algoritmo que: 
* a gestão de filas de espera em hospitais envolve a classificação de risco e priorização dos pacientes;
* monitorar a quantidade de pessoas inscritas e o tempo de espera;
* definir a prioridade de atendimento. Cada cor tem um tempo de espera tolerável, que indica a gravidade do caso:
    * Vermelho: risco de morte, atendimento imediato;
     * Laranja: tempo de espera tolerável de até 10 minutos;
    * Amarelo: tempo de espera tolerável de até 20 minutos;
    * Verde: não há risco de morte, tempo de espera tolerável de até 60 minutos;
    * Azul: quadros crônicos, encaminhamento ao centro de saúde, tempo de espera tolerável de até 180 minutos.
* organize a agenda de pacientes, devendo considerar:
    * o número de consultas marcadas;
    * o tempo médio de duração e a previsão de contratempos.

Em observação à lei abaixo, esta será considerada na implementação do algoritmo:
* A Lei 10.048/00 garante atendimento prioritário a idosos com 60 anos ou mais na administração pública.
* Também inclui pessoas do seguinte perfil: deficientes, gestantes, lactantes, pessoas com crianças de colo e obesos.
* Todavia, indivíduos com idade 80+ tem preferência sob todos os demais.

In [1]:
# !pip install faker
from faker import Faker
import random

import time

import pandas as pd
import datetime

### Gerando dados para teste

In [2]:
# Inicializando o Faker e configurando para nomes em português
fake = Faker("pt_BR")

# Inicializando as listas
lista_nomes = []          # string: primeiro nome ou nome completo/nome social
lista_dt_nascimento = []  # date: a data de nascimento
lista_prioridade = []     # bool: False para NÃO; True para SIM
lista_dt_entrada = []     # datetime: indica o horário de cadastro
lista_especialidade = []  # string: especialidade médica (geral, neurologia, oftalmologista etc)
lista_tma = []            # int: minutos com o tempo médio de atendimento para cada especialidade
lista_class_risco = []    # int: [0:1] representando azul, verde, laranja, amarelo, vermelho

lista_colunas = ['Nome', 'Nascimento', 'Prioridade', 'Entrada', 'Especialidade', 'TMA', 'Risco']
df = pd.DataFrame(columns=lista_colunas)

# Opções para os campos
especialidades_exemplo = ["geral", "neurologia", "oftalmologia", "cardiologia", "dermatologia", "pediatria"]
#class_risco_exemplo = ["vermelho", "laranja", "amarelo", "verde", "azul"]
class_risco_exemplo = [4, 3, 2, 1, 0]
# Função para gerar uma data de nascimento aleatória
def gerar_data_nascimento():
    start_date = datetime.date(1940, 1, 1)
    end_date = datetime.date(2005, 12, 31)
    return start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))


# Função para gerar uma data de entrada aleatória com horário
def gerar_data_entrada(is_random=True):
    now = datetime.datetime.now()
    if is_random:        
        delta_days = random.randint(0, 0)
        delta_seconds = random.randint(0, 86400)  # Número de segundos no dia
        return now - datetime.timedelta(days=delta_days, seconds=delta_seconds)
    else: return now

    
# Populando as listas com 100 registros
for _ in range(100):
    time.sleep(0.05)
    lista_nomes.append(fake.first_name())  # Gera um primeiro nome aleatório em português
    lista_dt_nascimento.append(gerar_data_nascimento())
    lista_prioridade.append(random.choice([True, False]))
    lista_dt_entrada.append(gerar_data_entrada(False))
    lista_especialidade.append(random.choice(especialidades_exemplo))
    lista_tma.append(random.choice([15, 30, 45, 60]))  # Tempo médio de atendimento entre 10 e 60 minutos
    lista_class_risco.append(random.choice(class_risco_exemplo))
    
lista_dados = [lista_nomes, lista_dt_nascimento, lista_prioridade, lista_dt_entrada,
               lista_especialidade, lista_tma, lista_class_risco]
    
for i, coluna in enumerate(lista_colunas):
    df[coluna] = lista_dados[i]

df

,Nome,Nascimento,Prioridade,Entrada,Especialidade,TMA,Risco
0,Valentina,1980-04-25,False,2024-11-01 16:20:18.701942,dermatologia,15,2
1,Otto,1955-10-30,False,2024-11-01 16:20:18.752605,geral,15,3
2,Breno,1978-09-07,True,2024-11-01 16:20:18.803270,pediatria,30,0
3,Léo,2000-06-03,True,2024-11-01 16:20:18.853952,neurologia,15,4
4,Leonardo,1966-02-02,False,2024-11-01 16:20:18.904606,dermatologia,60,3
...,...,...,...,...,...,...,...
95,Theo,1992-07-27,True,2024-11-01 16:20:23.523254,cardiologia,15,2
96,Vicente,1989-11-19,False,2024-11-01 16:20:23.573823,pediatria,15,4
97,Isabella,2005-03-13,True,2024-11-01 16:20:23.624402,pediatria,15,0
98,Luiz Henrique,1978-07-02,False,2024-11-01 16:20:23.674721,neurologia,60,2


### Iniciando funções e classes

In [32]:
class Fila():
    total_tma = 0
    
    def __init__(self):
        self.data = []

    def inserir(self, usuario, indice=''):
        if indice == '': self.data.append(usuario)
        else: self.data.insert(indice, usuario)
        self.total_tma+= usuario['TMA']

    def remover(self, indice=''):
        if len(self.data) > 0:
            if type(indice) is not int: usuario = self.data.pop(0)
            else: usuario = self.data.pop(indice)            
            self.total_tma-= usuario['TMA']
            return usuario

    def topo(self):
        if len(self.data) > 0:
            return self.data[0]

    def vazio(self):
        return not len(self.data) > 0

    def quantidade(self):
        return len(self.data)

    def exibir(self):
        return list(self.data)
    
    def tempo_espera(self):
        return self.total_tma
        
        
    
    
def calc_idade(nascido):
    """
    Calcula a idade de um cliente com base na data de nascimento fornecida.

    A função calcula a idade atual do cliente subtraindo o ano de nascimento do
    ano atual. Ajusta a idade caso o mês e o dia de nascimento ainda não tenham
    ocorrido no ano atual.

    Parâmetros:
    - nascido (datetime.date): A data de nascimento do cliente.

    Retorna:
    - int: A idade do usuário em anos.
    """
    
    now = datetime.datetime.now()
    return now.year - nascido.year - ((now.month, now.day) < (nascido.month, nascido.day))



def inserir_usuario(usuario):
    '''recebe obj usuario: dict ou pandas.series, acrescenta idade e senha, 
    extratifica conforme a idade e insere na fila respectiva'''
        
    idade = calc_idade(usuario['Nascimento'])
    usuario['Idade'] = idade    
    senha = str(usuario['Especialidade'][:5]+'-'+str(row[0])).upper()    
    
    if idade >= 80:
        senha = 'PE-'+senha    # obs.:PE > Prioridade Extra
        nome_fila = 'Prioridade 2'
        pos = manejar_risco(fila_priori2, usuario)
        
    elif idade >= 60 or usuario['Prioridade']:
        senha = 'P-'+senha
        nome_fila = 'Prioridade 1'
        pos = manejar_risco(fila_priori1, usuario)
        
    else:
        nome_fila = 'Normal'
        pos = manejar_risco(fila_normal, usuario)
        
    usuario['Senha'] = senha

    print('Usuário {}, senha: {}. Foi inserido na posição {} da fila {}'
          .format(usuario['Nome'], usuario['Senha'], pos, nome_fila))
    
    
    
def manejar_risco(fila, usuario_novo):
    '''fila: obj Fila(), usuario_novo: dict ou pandas.series, 
    seleciona a posição a ser inserida na fila, conforme o grau de risco'''
    
    risco_usuario_novo = usuario_novo['Risco']
    pos = 0
    if risco_usuario_novo < 5 or type(risco_usuario_novo) is not int:
        if not fila.vazio():
            for i in range(fila.quantidade() - 1, -1, -1):
                usuario_fila = fila.exibir()[i]
                if risco_usuario_novo <= usuario_fila.Risco:
                    pos = i + 1
                    fila.inserir(usuario_novo, pos)
                    break
                elif risco_usuario_novo > usuario_fila.Risco and i != 0:
                    continue
                elif i == 0:
                    pos = i
                    fila.inserir(usuario_novo, pos)            
        else: fila.inserir(usuario_novo)
        return pos
    else: print('Erro')

Há duas, principais, alternativas: 
1. Trabalhar com uma única fila, e dentro desta operar inserções e remanejos conforme necessário para adequar aos três graus de prioridades.
2. Fazer uso de três filas, uma para cada prioridade, mais intuitivo e fácil de implementar.

Neste primeiro momento, acabarei optando pela segunda alternativa.

In [33]:
# iniciando cada objeto fila
fila_normal = Fila()
fila_priori1 = Fila()
fila_priori2 = Fila()

In [34]:
# simulando a inserção de vários registros, em cada fila baseado na PREFERENCIA
for row in df.iterrows():
    user = row[1].copy()
    inserir_usuario(user)

Usuário Valentina, senha: DERMA-0. Foi inserido na posição 0 da fila Normal
Usuário Otto, senha: P-GERAL-1. Foi inserido na posição 0 da fila Prioridade 1
Usuário Breno, senha: P-PEDIA-2. Foi inserido na posição 1 da fila Prioridade 1
Usuário Léo, senha: P-NEURO-3. Foi inserido na posição 0 da fila Prioridade 1
Usuário Leonardo, senha: DERMA-4. Foi inserido na posição 0 da fila Normal
Usuário Enzo, senha: P-DERMA-5. Foi inserido na posição 2 da fila Prioridade 1
Usuário Maria Eduarda, senha: P-CARDI-6. Foi inserido na posição 2 da fila Prioridade 1
Usuário Laura, senha: P-DERMA-7. Foi inserido na posição 4 da fila Prioridade 1
Usuário Isaque, senha: P-GERAL-8. Foi inserido na posição 1 da fila Prioridade 1
Usuário Ágatha, senha: GERAL-9. Foi inserido na posição 2 da fila Normal
Usuário Erick, senha: P-NEURO-10. Foi inserido na posição 2 da fila Prioridade 1
Usuário Bruno, senha: OFTAL-11. Foi inserido na posição 0 da fila Normal
Usuário Manuella, senha: P-CARDI-12. Foi inserido na posi

### Quantidade em cada fila

In [6]:
fila_normal.quantidade()

35

In [7]:
fila_priori1.quantidade()

57

In [8]:
fila_priori2.quantidade()

8

In [87]:
fila_normal.exibir()

[Nome                              Ana Lívia
 Nascimento                       1968-12-19
 Prioridade                            False
 Entrada          2024-10-30 09:45:19.151522
 Especialidade                     pediatria
 TMA                                      45
 Risco                                     4
 Idade                                    55
 Senha                              PEDIA-48
 Name: 48, dtype: object,
 Nome                                 Sophie
 Nascimento                       1996-10-05
 Prioridade                            False
 Entrada          2024-10-30 09:45:20.776604
 Especialidade                         geral
 TMA                                      60
 Risco                                     4
 Idade                                    28
 Senha                              GERAL-80
 Name: 80, dtype: object,
 Nome                                  Laura
 Nascimento                       1990-03-22
 Prioridade                            False
 En

### Inserção de único usuário

In [40]:
# simulando as inserções na fila, com ajustes baseado no GRAU DE RISCO
# aviso: em algum momento, as inserções DEVEM respeitar alguma medida de tempo,
# caso contrário riscos de grau risco 4 (azul), serem atendidos muito tardiamente ou não serem atendidos

dict_user = {'Nome': 'Avril Lavigne',
             'Nascimento': datetime.date(1984, 9, 27),
             'Prioridade': False,
             'Entrada': gerar_data_entrada(False),
             'Especialidade': 'dermatologia',
             'TMA': 30,
             'Risco': 3}

inserir_usuario(dict_user)

Usuário Avril Lavigne, senha: DERMA-99. Foi inserido na posição 17 da fila Normal


In [41]:
user = fila_normal.exibir()[17]
user['Nome'], user['Idade'], user['Prioridade']

('Avril Lavigne', 40, False)

In [42]:
fila_normal.exibir()[17]['Nascimento']

datetime.date(1984, 9, 27)

In [44]:
# remoção de um usuário à fila
_ = fila_normal.remover(17)
_

{'Nome': 'Avril Lavigne',
 'Nascimento': datetime.date(1984, 9, 27),
 'Prioridade': False,
 'Entrada': datetime.datetime(2024, 11, 1, 16, 46, 40, 428028),
 'Especialidade': 'dermatologia',
 'TMA': 30,
 'Risco': 3,
 'Idade': 40,
 'Senha': 'DERMA-99'}

In [45]:
# tempo de espera em minutos
fila_normal.tempo_espera()

1245

#### Possibilidades e melhorias a serem feitas

* evitar dados duplicados em instâncias de objeto Fila()
* melhor geração de senha para guichê
* fornecer posiçao do usuário na fila
* para cada especialidade, poderia haver um critério como "grau de complexidade"

fontes:
    https://www.camara.leg.br/noticias/630767-PESSOAS-MAIS-IDOSAS-PODERAO-TER-PRIORIDADE-NO-ATENDIMENTO